In [ ]:
!which python; python -V;
import time

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.ephem import Ephem

from perylune.orbit_tools import *
from perylune.mpc import *

In [ ]:
# First step is to load the data. parse_txt() will load the orbital elements from text file in MPC format.
# This will be returned as an array. Each entry in that array has a list of 8 parameters or so.
elements = parse_txt("../data/mpc/mpcorb_extended.dat", limit=100, skip="------------------")

print("Number of orbital elements parsed: %d" % len(elements))

In [ ]:
# Now process the data. Find min and max a and e, also put the a values in 
max_e = 0
min_e = 9999999
max_a = 0
min_a = 9999999

a_tbl = np.zeros(len(elements))
e_tbl = np.zeros(len(elements))
cnt = 0
for e in elements:
    a = e[8].value
    ecc = e[6].value
    if a < 6:
        a_tbl[cnt] = a
        e_tbl[cnt] = ecc
        cnt = cnt + 1
    if max_a < a:
        max_a = a
    if min_a > a:
        min_a = a
    if max_e < ecc:
        max_e = ecc
    if min_e > ecc:
        min_e = ecc

print ("Processed %d asteroids, a=%f..%f [AU], eccentricity = %f..%f" % (cnt, min_a, max_a, min_e, max_e))

In [ ]:
# First, let's take a look at the major semiaxis. As seen here, the majority of the asteroids are in the main asteroid belt.
# The histogram is truncated at 6 AU. There are some TNO (Trans-Neptunian Objects) and KBO (Kuiper Belt Objects) that 
# are very far away
import plotly.graph_objects as go

fig = go.Figure(data=[go.Histogram(x=a_tbl)])
fig.update_layout(title_text='Distribution of %d asteroids\' semi major-axis' % len(a_tbl), xaxis_title_text='Major semiaxis (AU)', yaxis_title_text='Count')
fig.show()

In [ ]:
# Now do the eccentricity 
fig2 = go.Figure(data=[go.Histogram(x=e_tbl)])
fig2.update_layout(title_text='Solar System Asteroids (%d)' % len(e_tbl), xaxis_title_text='Eccentricity []', yaxis_title_text='Count')
fig2.show()

In [ ]:
import plotly.express as px
fig3 = px.scatter(x=a_tbl, y=e_tbl, labels={'x': 'Semi major axis [AU]', 'y': 'eccentricity'})
fig3.update_traces(marker_size=1)
fig3.show()